# 编码器-解码器架构

编码器

In [1]:
from torch import nn

# 继承自 nn.Module，符合PyTorch模型标准。作为抽象基类，不能直接被实例化使用
class Encoder(nn.Module):
    """编码器-解码器架构的基本编码器接口"""
    def __init__(self, **kwargs):
        '''
        接收任意关键字参数（如vocab_size,embed_size等）
        调用父类nn.Module的初始化器，确保参数注册、设备迁移等机制正常工作
        super(Encoder, self)获取Encoder类的父类（即nn.Module）
        .__init__(**kwargs)调用父类的初始化构造函数
        **kwargs将所有关键字参数原样传递给父类
        必须调用的原因:
        所有PyTorch模型必须继承nn.Module，而nn.Module的__init__方法做了关键初始化:
        注册参数：让.parameters()能遍历所有权重
        注册子模块：让.to(device)能迁移整个模型到GPU
        初始化钩子：设置.train() / .eval()模式切换机制
        '''
        super(Encoder, self).__init__(**kwargs)

    def forward(self, X, *args):
        '''
        强制子类实现：直接调用会抛出错误
        定义接口规范：所有子类必须实现forward方法，接受输入X和可选参数
        多态支持：后续EncoderDecoder类可以接收任意Encoder子类实例
        在基类中占位，这个方法只是接口定义，不能直接使用，必须由子类提供具体实现
        '''
        raise NotImplementedError

解码器

In [ ]:
'''
1. 继承结构
继承 nn.Module，符合PyTorch模型规范
抽象基类：不能直接实例化，必须由具体解码器（如Seq2SeqDecoder、TransformerDecoder）实现
'''
class Decoder(nn.Module):
    """编码器-解码器架构的基本解码器接口"""
    # 2. 初始化方法
    def __init__(self, **kwargs):
        # 调用父类构造函数，确保参数注册、设备迁移等功能正常
        super(Decoder, self).__init__(**kwargs)
    
    def init_state(self, enc_outputs, *args):
        '''
        3. 状态初始化方法
        这是解码器特有的接口，用于接收编码器输出并初始化解码器状态：
        enc_outputs:编码器的输出（通常是(output, state)元组）
        *args: 可选额外参数（如注意力机制的上下文）
        '''
        raise NotImplementedError
    # 4. 前向传播接口
    def forward(self, X, state):
        '''
        X:解码器输入（如目标语言词元，形状(batch,num_steps)）
        state:解码器状态（由init_state初始化，会在每个时间步更新）
        返回:(output,new_state)
        '''
        raise NotImplementedError

合并编码器和解码器

In [ ]:
'''
与Encoder/Decoder类的关系
Encoder/Decoder (抽象接口)
       ↓
Seq2SeqEncoder/Seq2SeqDecoder (具体实现)
       ↓
EncoderDecoder (组合封装)
EncoderDecoder是 "组合模式"：持有编码器和解码器实例，协调它们的工作
它本身不实现具体逻辑，而是委托给子模块
'''
# 继承nn.Module，遵循PyTorch模型标准，具体实现类（非抽象类），可直接实例化使用
class EncoderDecoder(nn.Module):
    """编码器-解码器架构的基类"""
    def __init__(self, encoder, decoder, **kwargs):
        '''
        encoder: 任意Encoder子类实例（如 Seq2SeqEncoder）
        decoder: 任意Decoder子类实例（如 Seq2SeqDecoder）
        注册子模块：将编码器和解码器注册为self的属性，使其参数能被.parameters()捕获，并能随模型一起.to(device)
        '''
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    '''
    编码阶段：enc_X（源语言序列） →encoder→enc_outputs:通常enc_outputs是元组 (output, state)
    状态初始化：enc_outputs→decoder.init_state()→dec_state:将编码器最终状态转换/传递给解码器
    解码阶段：dec_X（目标语言序列） +dec_state→decoder→最终输出
    *args的作用：透传额外参数（如序列长度、注意力掩码等）给编码器和解码器
    '''
    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)